Team Members:
- Emily Liang 79453973
- Kristen Chung 42617410
- 
-

In [ ]:
# Install required packages
%pip install gradio unstructured sentence-transformers
%pip install google.generativeai     # for using local IDE

In [ ]:
import sys
# print(sys.executable)
!{sys.executable} -m pip install google-generativeai

In [3]:
# Import necessary libraries (RERUN THIS CELL IF NEEDED)
import os
import time
from PIL import Image
import google.generativeai as genai
import gradio as gr 

In [4]:
from google.generativeai import types #(RERUN THIS CELL IF NEEDED)
import os

# For local environment, directly put your API key here
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)
model = genai.GenerativeModel(model_name="gemini-2.5-flash-lite")

In [5]:
# PART 1: Transcribing handwritten Python code into a code snippet - Emily

import re

# Goal: take an img including handwritten python code, feed it into LLM, & return the python code as text
def transcribe_image_code(image_file: Image.Image) -> str:
    """
    Step 1: Transcribes code from the input image using a multimodal LLM.

    Parameters:
        image_file (Image.Image): The uploaded image containing handwritten code.

    Returns:
        str: The transcribed code as a string.
    """
    
    # prompt given to the LLM to transcribe the code from the image
    prompt = """Given the following image containing handwritten Python code, transcribe the code accurately into a text format and output it."""

    # call the LLM with the prompt and image
    output = model.generate_content([prompt, image_file])

    # extract the transcribed code from the LLM output & ensure it has text
    code = output.text if output and output.text else ""

    # remove any markdown formatting (like ```python ... ```) from the transcribed code
    code = re.sub(r"```(?:python)?\n?", "", code) 
    code = code.replace("```", "")

    # return the cleaned code as a string
    return code.strip()

# Testing the function with an example image
img_path = "code_images_example/code_image_07.jpg"
image = Image.open(img_path)
print(transcribe_image_code(image))


E0000 00:00:1761896679.467766 4301318 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


def flatten(arr):
    for x in arr:
        if isinstance(x, list):
            for y in flatten(x):
                yield y
        else:
            yield x


In [14]:
# PART 2: Running static analysis and explaining bugs (if any) in natural language - Kristen
import json
import re
import gradio as gr
import google.generativeai as genai



def analyze_code(code_block: str):
    """
    Step 2: Performs static analysis on the transcribed code.

    Parameters:
        code_block (str): The transcribed code snippet.

    Returns:
        (1) The analysis text explaining bugs, improvements, or efficiency suggestions.
        (2) The refined version of the code snippet with the suggested fixes and enhancements.
    """

    if not code_block:
        raise gr.Error("Please provide a valid code snippet.")

    try:
        model = genai.GenerativeModel(model_name="gemini-2.5-flash-lite")

        prompt = f"""
        You are an expert Python developer and code reviewer.
        Analyze the following code snippet and do the following:
        1. Explain what the code does in natural language.
        2. Identify any syntax or logical bugs.
        3. Suggest specific fixes and efficiency improvements.
        4. Provide a refined version of the code with your fixes applied.

        Output your response in the following JSON format:
        {{
            "analysis": "detailed explanation and improvements",
            "refined_code": "refined Python code"
        }}

        Code to analyze:
        ```
        {code_block}
        ```
        """
        response = model.generate_content(prompt)
        text = (response.text or "").strip()


        try:
            data = json.loads(text)
        
        except json.JSONDecodeError:
            # format 
            text = text.replace("```json", "").replace("```", "").strip()
            data = json.loads(text)

        analysis = data.get("analysis", "No analysis found.")
        refined = data.get("refined_code", code_block)

        return analysis, refined

    except Exception as e:
        raise gr.Error(f"Error during analysis: {e}")

# test call
img_path = "code_images_example/code_image_08.jpg"
image = Image.open(img_path)
transcribed_code = transcribe_image_code(image)
analysis, refined = analyze_code(transcribed_code)
print(analysis)
print(refined)

1. **What the code does:**
This Python function `flatten` is intended to take a nested list (a list that may contain other lists within it) and return an iterator that yields all the non-list elements from the nested structure in a flattened manner. It uses recursion to traverse through the nested lists.

2. **Syntax or logical bugs:**
   - **Logical Bug 1: Infinite Recursion:** The most critical bug is in the `else` block: `yield flatten(x)`. When `x` is not a list, it's assumed to be a non-list element. However, the code incorrectly calls `flatten(x)` again. If `x` is an integer, for example, `flatten(x)` will raise a `TypeError` because `isinstance(x, list)` will be `False`, and then the code will attempt to iterate over `x` (which is an integer), leading to an infinite recursion or a `TypeError` depending on the exact type of `x`.
   - **Logical Bug 2: Incorrect Yielding of Non-List Elements:** In the `else` block, instead of yielding the element `x` itself, it yields the *result* 

In [ ]:
# PART 3: Suggesting bug fixes, improvements, or efficiency tweaks to the code snippet

"""
Example of running test cases on LLM-generated code.
You do not need to follow this exact implementation for your code.
"""
import json
import importlib.util

def run_tests(filename_original, function_name, json_test_path):
    # Load test cases
    with open(json_test_path, 'r') as f:
        test_cases = json.load(f)["test_case"]

    # Load the function from the file
    spec = importlib.util.spec_from_file_location(function_name, filename_original)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    func = getattr(module, function_name)

    # Run tests
    for idx, case in enumerate(test_cases):
        try:
            inputs = case["input"]
            expected = case["expected"]
            result = func(*inputs) if isinstance(inputs, (list, tuple)) else func(inputs)
            assert result == expected, f"input={inputs}, expected={expected}, got={result}"
            print(f"Test {idx+1} passed.")
        except Exception as e:
            print(f"Test {idx+1} failed: {e}")

"""
Make sure to save the final code (after transcribing and performing static analysis) into a Python file.
For example, if you have saved the final code transcribed and fixed by the LLM as example_llm_code.py,
you can run the test cases using the format below.

You can also add more inputs and expected outputs to the JSON file to run additional tests.
It is encouraged to add more test cases to ensure the robustness of your code.
"""

run_tests(
    filename_original="example_llm_code.py",
    function_name="bucketsort",
    json_test_path="test_case_bucketsort.json"
)

In [ ]:

# PART 4: Implement the Gradio Interface to run the app

if __name__ == "__main__":
    app.launch(share=True)